In [1]:
# import pickle
# import os
# import matplotlib.pyplot as plt
# import sklearn.tree as tree

# # Load the decision tree model
# with open(os.path.join("dt_model", "decision_tree_depth_8.pkl"), "rb") as f:
#     dt_model = pickle.load(f)

# # Print depth
# print(f"Decision Tree Depth: {dt_model.get_depth()}")

# # Load the saved feature names
# with open(os.path.join("dt_model", "feature_names.pkl"), "rb") as f:
#     feature_names = pickle.load(f)

# # Verify dimensions match
# if dt_model.n_features_in_ == len(feature_names):
#     print(f"Found {len(feature_names)} feature names, matching model dimensions")
# else:
#     print(f"Warning: Model has {dt_model.n_features_in_} features but found {len(feature_names)} feature names")
#     # Fall back to generic names if needed
#     feature_names = [f"feature_{i}" for i in range(dt_model.n_features_in_)]

# # Plot the tree
# plt.figure(figsize=(20, 15))
# tree.plot_tree(dt_model, 
#               feature_names=feature_names,
#               class_names=['action_0', 'action_1'],
#               filled=True,
#               max_depth=None)
# #plt.savefig("decision_tree_viz.png", dpi=300, bbox_inches='tight')
# plt.show()

In [5]:
import pyshark
import pandas as pd
import os 

def get_out_of_order_packets(pcap_file, dst_ip, dst_port):
    """
    Extract out-of-order TCP packets received by the server from pcap.

    Args:
        pcap_file (str): Path to server-side pcap file.
        dst_ip (str): IP address of the receiving server.
        dst_port (int or str): Port number of the server.

    Returns:
        pd.DataFrame: DataFrame containing details of out-of-order packets.
    """
    print(f"Processing PCAP: {pcap_file}")
    capture = pyshark.FileCapture(
        pcap_file,
        display_filter=f'tcp.dstport=={dst_port} && ip.dst=={dst_ip}',
        use_json=True
    )

    # Data structure to track highest seen sequence number per flow
    flows_seq = {}
    out_of_order_pkts = []

    for pkt in capture:
        try:
            tcp = pkt.tcp
            ip = pkt.ip

            flow_key = (
                ip.src, int(tcp.srcport), ip.dst, int(tcp.dstport)
            )

            seq_num = int(tcp.seq)
            seq_len = int(tcp.len)  # TCP payload length

            if flow_key not in flows_seq:
                flows_seq[flow_key] = seq_num + seq_len
                continue

            expected_seq = flows_seq[flow_key]

            if seq_num < expected_seq:
                # Out-of-order packet detected
                out_of_order_pkts.append({
                    'timestamp': pkt.sniff_time,
                    'src_ip': ip.src,
                    'src_port': int(tcp.srcport),
                    'dst_ip': ip.dst,
                    'dst_port': int(tcp.dstport),
                    'sequence_number': seq_num,
                    'expected_sequence_number': expected_seq,
                    'payload_length': seq_len
                })
            else:
                # Update highest sequence seen
                flows_seq[flow_key] = seq_num + seq_len

        except AttributeError:
            # Skip non-TCP packets or incomplete packets
            continue

    capture.close()

    print(f"Total out-of-order packets found: {len(out_of_order_pkts)}")
    return pd.DataFrame(out_of_order_pkts)


server_ip = '10.0.2.2'  
server_port = 12345 
exp_id = '20250328_162243'
pcap_file = os.path.join('tmp', exp_id, f'background_server_{server_ip}_{server_port}.pcap')

ooo_packets_df = get_out_of_order_packets(pcap_file, server_ip, server_port)

# Save results to CSV
ooo_packets_df.to_csv('out_of_order_packets.csv', index=False)

print(ooo_packets_df.head())


Processing PCAP: tmp/20250328_162243/background_server_10.0.2.2_12345.pcap


RuntimeError: This event loop is already running